In [1]:
from pinecone import Pinecone
import os
from dotenv import load_dotenv

load_dotenv()
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
index = pc.Index(host=os.getenv("PINECONE_HOST"))

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(os.getenv("MODEL_NAME"))
model = AutoModelForCausalLM.from_pretrained(
    os.getenv("MODEL_NAME"), device_map="auto", dtype="auto"
)

In [3]:
def answer_query_with_context(query: str, top_k: int = 3, max_tokens: int = 200):
    # Pinecone search
    results = index.search(
        namespace=os.getenv("PINECONE_NAMESPACE"),
        query={"inputs": {"text": query}, "top_k": top_k},
        fields=["text", "source", "category"]
    )
    
    # Extracting text from hits
    hits = results["result"]["hits"]
    context = "\n".join([hit["fields"]["text"] for hit in hits if "fields" in hit and "text" in hit["fields"]])
    if not context:
        context = "No relevant information found in the knowledge base."
    
    # Prompt
    prompt = (
        f"Answer the question using the context below. "
        f"Answer as accurately as possible based on the context.\n\n"
        f"Context:\n{context}\n\nQuestion: {query}\nAnswer:"
    )
    
    # Tokenization and generation
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=max_tokens)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return answer

In [4]:
user_query = "What is your name and what are your hobbies?"
answer = answer_query_with_context(user_query)
print(answer)

Answer the question using the context below. Answer as accurately as possible based on the context.

Context:
I have several hobbies such as playing bass guitar, swimming, playing video games, I go to the gym, I run sometimes, I sing, I read books, and many more.

I like to explain technical concepts clearly and concisely, often with examples. I try to make my explanations friendly, approachable, and a bit playful when possible.


My full name is Ramon Emmiel P. Jasmin. The "P" is my middle name which is Palcon. The "Ramon" came from my father's name which is "Vic Ramon". The "Emmiel" came from my grandmother's name which is "Emerita"


Question: What is your name and what are your hobbies?
Answer: 
Hi there! I'm Ramon Emmiel P. Jasmin, but you can call me Ramon for short. I'm a pretty laid-back guy with a lot of interests, so I'll give you the rundown of my hobbies.

First off, I'm a bit of a music enthusiast. I play the bass guitar and sing in a band. I also enjoy playing video games